In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np

In [3]:
# !pip install transformers==2.1.0
!pip install gensim==4.1.2
!pip3 install --user underthesea
!pip install tensorflow_addons
# !pip install pyvi
!pip install transformers
!pip install fastBPE
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 19 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 5.1 MB/s 
     |████████████████████████████████| 965 kB 80.2 MB/s 
     |████████████████████████████████| 581 kB 98.6 MB/s 
     |████████████████████████████████| 235 kB 102.5 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████

In [1]:
#from pyvi import ViTokenizer, ViPosTagger
import regex as re
from underthesea import word_tokenize
#from vncorenlp import VnCoreNLP
import numpy as np
import pandas as pd
import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
#from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

def remove_webURL(txt):
  return re.sub(r'https?://www\.\S+.com','',txt)

def remove_emoji(txt):
  emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F" #emoticons
    u"\U0001F300-\U0001F5FF"#symbols
    u"\U0001F680-\U0001F6FF"#transport
    u"\U0001F1E0-\U0001F1FF"#flags
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', txt)

def remove_multiplechars(txt):
  txt = re.sub(r'(.)\1{3,}',r'\1', txt)
  return txt

In [3]:
def text_preprocess(document):
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # đưa về lower
    document = document.lower()

    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)

    # xoa web URL
    document = remove_webURL(document)
    
    # xoa emoji
    document = remove_emoji(document)

    # xóa html code
    document = remove_html(document)
    
    # xoa ki tu lap lai
    document = remove_multiplechars(document)
    
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    # tách từ
    document = word_tokenize(document, format="text")
    # document = rdrsegmenter.tokenize(document)
    # document = ' '.join(document)
    return document


In [4]:
# lst_stopword = []
# with open('/content/drive/MyDrive/KhanhMD/Data/vietnamese-stopwords.txt','r') as f:
#     for line in f:
#         for word in line.split():
#            lst_stopword.append(word)  
    
# def remove_stopwords(line):
#     words = []
#     for word in line.strip().split():
#         if word not in lst_stopword:
#             words.append(word)
#     return ' '.join(words)

In [5]:
with open('/content/drive/MyDrive/KhanhMD/Data/data_pos.txt', 'r') as file:
    data_pos = file.read().replace('\n', '/27121998/')

In [6]:
with open('/content/drive/MyDrive/KhanhMD/Data/data_neg.txt', 'r') as file:
    data_neg = file.read().replace('\n', '/27121998/')

In [7]:
clean_data_pos = text_preprocess(data_pos)

In [8]:
clean_data_neg = text_preprocess(data_neg)

In [9]:
data_pos[:100]

'Đây là nhà_thờ được cho là lớn nhất Huế , lại nằm gần_như ở trung_tâm thành_phố . Với lối kiến_trúc '

In [10]:
clean_data_neg[:100]

'mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi hơn 20 hỏi lại thì nv tl có r nhg bảo chờ thêm 15 nữa '

In [11]:
text_pos = clean_data_pos.split(' 27121998 ')

In [12]:
text_pos[:100]

['đây là nhà_thờ được cho là lớn nhất huế lại nằm gần_như ở trung_tâm thành_phố với lối kiến_trúc của người pháp nhà_thờ mang nét cổ_kính pha lẫn với hiện_đại rất châu_âu nên bản_thân mình thấy kiến_trúc nhà_thờ rất đẹp toát lên cái cao_quý của nhà_thờ không_gian ở đây khá yên_bình và mỗi dịp giáng_sinh về nơi đây luôn là nơi rực_rỡ và đẹp nhất',
 'cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà mới có dịp được ăn trước giờ toàn ăn cơm hến bún hến ở đường hải_phòng thôi có thời_gian lên ăn_ở chợ_phú_lộc hến béo hơn thơm hơn nhiều công_nhận nước_ngọt nữa gía chỉ 10 tô quá rẻ rồi',
 'resort ngay trung_tâm cách phố cổ tầm 700 m 1 km và bạn dễ_dàng đi bộ hoặc đạp xe vào phố phòng_ốc xinh_xắn đặc_biệt là hồ bơi hồ bơi không chứa clo mà thay bằng nước_biển lọc mặn spa được cộng trong tiền_phòng nên mỗi đêm ở bạn được cộng 1 buổi spa luôn nhé',
 'chỉ cách phổ cổ khoảng 1 km nên khách_sạn này rất tiện_lợi do mới xây nên rất mới đẹp phòng_ốc có loại phòng nhỏ và phòng bự có bồ sục mình ở loại phò

In [13]:
text_neg = clean_data_neg.split(' 27121998 ')
text_neg[:100]

['mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi hơn 20 hỏi lại thì nv tl có r nhg bảo chờ thêm 15 nữa tụi e lm liền mình k biết có ngon k nhg cũng muốn ăn thử thiết_nghĩ nv quán nên xem_lại cách pv và nc vs khách',
 'quán này khá là nổi_tiếng nay mới có dịp ghé thử năm ở vii trí khá dễ tìm quán hơi nhỏ đi chiều_tối là đông lắm phải đợi lâu nhân_viên thì bình_thường à vì do đông quá nên cũng thông_cảm món ăn mình gọi gà sốt phô_mai cay gà quá bở thịt ăn vào k thơm món này đã có mỡ rồi cộng thêm da_gà nữa ăn rất ngáy cảm_giác ngán lắm cơm trộn kim_chi hq thì_phải ăn hơi cay nóng_hổi luôn vị ăn ok nhưng k đặc_sắc với giá đó thì ăn k đáng tiền khoai_tây chiên dở và ăn vị k ngon tóm_lại k như kì_vọng của mình giá_cả ở đây quá đắt so với chất_lượng mà đi ăn phải chầu_chực như_thế thì cũng hơi mệt à có nước_lọc miễn_phí nha mọi người tới ăn nhớ hỏi',
 'thứ 6 nào ta cùng quẩy vuvuzela beer club chung hệ_thống gogi_house daruma kichi kichi nên giá cũng chưa 10 vat thanh_toán trước 20 g hằng ngà

In [14]:
import pandas as pd
df = pd.DataFrame(text_pos, columns=["text"])
df.to_csv('text_pos.csv', index=False)

In [15]:
text_pos_pd = pd.read_csv('text_pos.csv')
text_pos_pd[:10]

,text
0,đây là nhà_thờ được cho là lớn nhất huế lại nằ...
1,cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà ...
2,resort ngay trung_tâm cách phố cổ tầm 700 m 1 ...
3,chỉ cách phổ cổ khoảng 1 km nên khách_sạn này ...
4,hôm_nay mình quay lại quán cùng_với mấy người ...
5,bánh và nước rất rất ngon phục_vụ nhiệt_tình c...
6,hôm đấy ngẫu_hứng tới nồi vì đang có chương_tr...
7,nghe tên quán_từ rất lâu rồi đọc review cũng t...
8,mình đến nồi_bay hôm trước và có gọi bánh mous...
9,tổng_hợp từ 2 lần em đến với nồi_bay bình_thườ...


In [16]:
text_pos_pd=text_pos_pd.assign(label=lambda x: 1)

In [17]:
df_0 = pd.DataFrame(text_neg, columns=["text"])
df_0.to_csv('text_neg.csv', index=False)

In [18]:
text_neg_pd = pd.read_csv('text_neg.csv')
text_neg_pd[:10]

,text
0,mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi ...
1,quán này khá là nổi_tiếng nay mới có dịp ghé t...
2,thứ 6 nào ta cùng quẩy vuvuzela beer club chun...
3,mình đi với nhóm tổng_cộng 4 người ăn chỉ có k...
4,nhân_viên phục_vụ không mấy tận_tình đồ_ăn ra ...
5,vào đây thì hết bàn nhưng mình vẫn ngồi đợi bì...
6,hôm_nay đến aeon có ghé vào quán món ăn cũng k...
7,mình vào lúc 8 h tối cũng không đông khách lắm...
8,thức_ăn quá_đỗi bình_thường miếng xôi của bạn ...
9,lần đầu_tiên ăn_chắc cũng là lần cuối ăn_ở đây...


In [19]:
text_neg_pd=text_neg_pd.assign(label=lambda x: 0)

In [20]:
df_concat_col = pd.concat([text_pos_pd, text_neg_pd])
df_concat_col.to_csv("data.csv", index=False)

In [21]:
c = pd.read_csv("data.csv")
c[:10]

,text,label
0,đây là nhà_thờ được cho là lớn nhất huế lại nằ...,1
1,cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà ...,1
2,resort ngay trung_tâm cách phố cổ tầm 700 m 1 ...,1
3,chỉ cách phổ cổ khoảng 1 km nên khách_sạn này ...,1
4,hôm_nay mình quay lại quán cùng_với mấy người ...,1
5,bánh và nước rất rất ngon phục_vụ nhiệt_tình c...,1
6,hôm đấy ngẫu_hứng tới nồi vì đang có chương_tr...,1
7,nghe tên quán_từ rất lâu rồi đọc review cũng t...,1
8,mình đến nồi_bay hôm trước và có gọi bánh mous...,1
9,tổng_hợp từ 2 lần em đến với nồi_bay bình_thườ...,1


In [22]:
c = c.sample(frac=1).reset_index(drop=True)

In [23]:
df = c.copy()
df[:50]

,text,label
0,mình thích nhất mấy món chiên xào đồ các kiểu ...,1
1,lạp_xưởng này giống kiểu xúc_xích nhưng dai hơ...,0
2,cafe cộng phải nói là rất tuyệt cũng hay tới c...,1
3,trên cả tuỵêt vời menu mới có mon salat trái_c...,1
4,quán này gần trường nên mình hay đi ăn quán th...,0
5,cách trang_trí vẫn chưa đẹp mặc_dù có chút lấy...,0
6,đây là lần thứ 2 mình bị trường_hợp như_vậy mì...,0
7,phở nước_dùng đậm_đà đúng vị nước hầm xương_th...,1
8,mình và đứa bạn khá thích ăn đồ nướng 2 đứa ha...,0
9,so với các quán trà sữa hiện_nay thì không_gia...,0


In [24]:
df.shape

(49636, 2)

In [25]:
text = clean_data_pos +' '+ clean_data_neg

In [26]:
text = text.split(' 27121998 ')

In [27]:
split_text = [s.split() for s in text]

In [28]:
a = pd.DataFrame(df, columns = ['text'])
a

,text
0,mình thích nhất mấy món chiên xào đồ các kiểu ...
1,lạp_xưởng này giống kiểu xúc_xích nhưng dai hơ...
2,cafe cộng phải nói là rất tuyệt cũng hay tới c...
3,trên cả tuỵêt vời menu mới có mon salat trái_c...
4,quán này gần trường nên mình hay đi ăn quán th...
...,...
49631,tiệm_cơm_lăng_ký rộng thoáng mát sạch_sẽ đồ_ăn...
49632,phục_vụ chuyên_nghiệp nhah nhẹn ks đẹp y nhu t...
49633,không_gian rộng nhưng bố_trí khít quá làm cảm_...
49634,qua ́ n co ́ ca ́ c xiên chuâ ̉ n bi ̣ kha ́ n...


In [29]:
b = pd.DataFrame(df, columns = ['label'])
b

,label
0,1
1,0
2,1
3,1
4,0
...,...
49631,1
49632,1
49633,0
49634,1


In [30]:
text_lst = df['text'].tolist()
label_lst = df['label'].tolist()

In [31]:
text_lst[:10]

['mình thích nhất mấy món chiên xào đồ các kiểu cũng gần nhà đi ngang buổi tối thấy ko gian vô_cùng lãng_mạn với dàn đèn gió mát thích ngồi ở khu_vực ngoài_trời hơn vì ko gian ko bị gò_bó ko ngột_ngạt bánh_xèo ngon nhưng ko được giòn cho lắm viền bánh thì hơi nhạt chấm với nước_chấm ở đây thì cũg gọi_là tạm ổn mì xào giòn thì khỏi chê kkk nhiều tôm nhiều thịt và cũng nhiều mì mì xào giòn ko cứng mềm ăn good 2 người ăn muốn ná thở tính thử bánh khọt nữa nhưng_mà ko còn chổ chứa hiu_hiu',
 'lạp_xưởng này giống kiểu xúc_xích nhưng dai hơn 1 tẹo phải chờ hơi lâu hôm mình đi thì_có 1 e gái 1 e zai phục_vụ hơi chậm và lóng_ngóng điểm cộng là sốt quét khá ngon đặc_biệt là vị bbq',
 'cafe cộng phải nói là rất tuyệt cũng hay tới cộng để uồsng cafe với bạn_bè lắm hôm trước tới gọi thử 1 lần sữa_chua cafe thấy uống cũng ngon nhưng vẫn ko bằng cafe cốt dừa',
 'trên cả tuỵêt vời menu mới có mon salat trái_cây ngon cực_kì lun mình vừa_mới thử và cảm_thấy rất thích m n tơi dùg thử coi s mấy bạn nữ có

In [32]:
label_lst[:10]

[1, 0, 1, 1, 0, 0, 0, 1, 0, 0]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(text_lst, label_lst, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.9 x 0.8 = 0.2


In [34]:
# !wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
# !tar -xzvf PhoBERT_base_transformers.tar.gz

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [36]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/drive/MyDrive/KhanhMD/BERT/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/drive/MyDrive/KhanhMD/BERT/PhoBERT_base_transformers/dict.txt")

In [37]:
MAX_LEN = 200

train_ids = []
for sent in X_train:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in X_val:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)
    
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [38]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]
    val_masks.append(mask)

In [39]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)


train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)


train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)


train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=64)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=64)

In [40]:
train_ids = np.array(train_ids)
val_ids = np.array(val_ids)

y_train = np.array(y_train)
y_val = np.array(y_val)

In [41]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW,TFRobertaModel

config = RobertaConfig.from_pretrained(
    "/content/drive/MyDrive/KhanhMD/BERT/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 2, output_hidden_states=False,
)
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/KhanhMD/BERT/PhoBERT_base_transformers/model.bin",
    config=config
)

# BERT_TF = TFRobertaModel.from_pretrained(
#     "/content/PhoBERT_base_transformers/model.bin",
#     config=config,from_pt=True
# )

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/KhanhMD/BERT/PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifi

In [42]:
BERT_SA.cuda()
print('Done')

Done


In [43]:
import torch
torch.cuda.empty_cache()

In [44]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    F1_score = f1_score(pred_flat, labels_flat, average='macro')
    
    return accuracy_score(pred_flat, labels_flat), F1_score

In [45]:
import random
from tqdm import tqdm_notebook
device = 'cuda'
epochs = 3

param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)


for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_SA.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()



        
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        nb_train_steps += 1
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print(" Train Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" Train F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" Average Training loss: {0:.4f}".format(avg_train_loss))

    print("Running Validation...")
    BERT_SA.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    for batch in tqdm_notebook(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

            eval_accuracy += tmp_eval_accuracy
            
            eval_f1 += tmp_eval_f1
            nb_eval_steps += 1
    print(" Val Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" Val F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
    
print("Training complete!")

======== Epoch 1 / 3 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Train Accuracy: 0.8864
 Train F1 score: 0.8829
 Average Training loss: 0.2717
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Train Accuracy: 0.9385
 Train F1 score: 0.9375
 Average Training loss: 0.1696
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/156 [00:00<?, ?it/s]

 Val Accuracy: 0.9144
 Val F1 score: 0.9106
Training complete!


In [46]:
test_ids = []
for sent in X_test:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    test_ids.append(encoded_sent)

test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [47]:
test_masks = []
for sent in test_ids:
    mask = [int(token_id > 0) for token_id in sent]
    test_masks.append(mask) 

In [48]:
test_inputs = torch.tensor(test_ids)
test_labels = torch.tensor(y_test)
test_masks = torch.tensor(test_masks)

In [49]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=32)

In [50]:
test_ids = np.array(test_ids)
y_test = np.array(y_test)

In [51]:
BERT_SA.eval()
test_loss, test_accuracy = 0, 0
nb_test_steps, nb_test_examples = 0, 0
test_f1 = 0
for batch in tqdm_notebook(test_dataloader):

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = BERT_SA(b_input_ids, 
        token_type_ids=None, 
        attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_test_accuracy, tmp_test_f1 = flat_accuracy(logits, label_ids)

        test_accuracy += tmp_test_accuracy
        
        test_f1 += tmp_test_f1
        nb_test_steps += 1
print(" Test Accuracy: {0:.4f}".format(test_accuracy/nb_test_steps))
print(" Test F1 score: {0:.4f}".format(test_f1/nb_test_steps))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/311 [00:00<?, ?it/s]

 Test Accuracy: 0.9104
 Test F1 score: 0.9077
